In [1]:
# Load modules
import os
import glob
import cftime
import numpy as np
import xarray as xr
import pandas as pd
import netCDF4 as nc
from tqdm import tqdm
from scipy import signal
import matplotlib.pyplot as plt
import scipy.interpolate as interp

# Zonal Means

In [3]:
### Manual input ###
# Paths 
jpath     = '/projects/GEOCLIM/LRGROUP/datasets_2020_Jan/JRA55-do-v1.5merge/data_origin_v1.5/' # to JRA products
#spath     = '../../forcings/idealized_year_control/zonal_means/'  # save Zonal Mean
#spath     = '../../forcings/warming2deg_year/zonal_means/'  # save Zonal Mean
spath     = '../../forcings/idealized_year_cold_NHtemp/zonal_means/'  # save Zonal Mean


# Specify which forcings to average in JRA (based on data_table)
forcings  = ['psl', 'ts', 'huss', 'rlds', 'prra', 'prsn']

# Conversion from Kelvin to Celsius
k2C       = -273.15

# Temperature threshold for land mask (in Celsius; threshold does not make a huge difference)
tthresh  = -5

# Strings to load relevant JRA data
str90    = '*_199*'
str20    = '*_20*'
strYmask = '*_2019*'

# Strings for filename
strAvg   = '30yrs'
strReg   = 'basin'
strLand  = 'LandMask'

# Boundaries for averaging time periods (30 years)
t1      = pd.Timestamp(1990, 1, 1, 0)
t2      = pd.Timestamp(2020, 1, 1, 0)

# Time format in deposition files is different
tC1 = cftime.DatetimeNoLeap(1990, 1, 1, 0, 0, 0, 0, has_year_zero=True)
tC2 = cftime.DatetimeNoLeap(2020, 1, 1, 0, 0, 0, 0, has_year_zero=True)

# Boundaries for spatial averaging
# Note: land mask does not work for Africa so stay within ocean
lat1    = 15
lat2    = 65  
lon1    = 295
lon2    = 340

In [4]:
### Function to generate land mask ###
# Use temperatures at midnight on January 1, 2019
fpath  = glob.glob(jpath + 'ts' + strYmask)
dsR    = xr.open_mfdataset(fpath, combine='by_coords')

def land_mask(lat01, lat02, lon01, lon02):
    # Slice the dataset based on the area considered in the calculations
    dsS  = dsR.sel(lat=slice(lat01, lat02), lon=slice(lon01, lon02))
    mask = dsS.ts.isel(time=0).where(dsS.ts.isel(time=0)+k2C < tthresh, other = 1)  # set the ocean as 1
    mask = mask.where(dsS.ts.isel(time=0)+k2C >= tthresh, other = 0)  # set the land as 0
    return mask

In [ ]:
### Loop to calculate monthly zonal means ###
for iforc in tqdm(forcings):  # loop through each forcing
    
    # Load 30-years of data 
    fpath  = glob.glob(jpath + iforc + str90) + glob.glob(jpath + iforc + str20)
    ds     = xr.open_mfdataset(fpath, combine='by_coords')

    # Create the land mask
    mask = land_mask(lat1, lat2, lon1, lon2)

    # Calculate zonal average
    zAvg = ds[iforc].sel(lat=slice(lat1, lat2), 
                         lon=slice(lon1, lon2), 
                         time=slice(t1, t2)
                        ).where(mask).mean(dim='lon', skipna=True).compute()

    # Calculate monthly average
    mAvg = zAvg.groupby("time.month").mean(dim="time")

    # Close zAvg (help with memory?) 
    zAvg.close()

    # Save monthly zonal averages for further modifications
    fsname = '_zonal_JRA_clim' + strAvg + '_monthly_' + strReg + '_' + strLand + '.nc'
    mAvg.to_netcdf(spath + iforc + fsname)

    # Close mAvg (help with memory?) 
    mAvg.close()
            
    ds.close()  # close for the forcing

# Gridded

In [2]:
# Manual input
# Paths
spath     = '../../forcings/idealized_year_control/zonal_means/'  # to saved zonal means
gpath     = '../../forcings/idealized_year_cold_NHtemp/gridded/'  # save gridded forcing directly into INPUT folder   

# Filenames in and out
fnameLF_in  = '_zonal_JRA_clim30yrs_monthly_basin_LandMask.nc'
fnameLF_out = '_JRA_clim30yrs_monthly_basin_LandMask_gridded.nc'

# Specify which forcings to adjust
forcings = ['psl', 'ts', 'huss', 'rlds', 'prra', 'prsn']
scaledF  = ['prra', 'prsn']  # scale precipitation by domain area vs. real ocean

# Lat and lon of forcing domain
hres     = 0.5
lat_out  = np.arange(15, 65.5, hres)  # from 15 to 65
lon_out  = np.arange(300, 350.5, hres)  # from 300 to 350

# Fill value
fillval  = -1e+20  # set fill value (Raphael/Enhui: -1e+20; Elizabeth:-1.e+34)

# Create a 12-month time vector 
time_m = pd.date_range(start = '1990-01-01', periods=12, freq = 'MS')+ pd.DateOffset(days=14)
# Enhui's file has days since 1990-01-01
time_m = np.asarray(time_m.dayofyear, dtype=np.float64)-1

# Conversion from kelvin to celsius
k2C      = -273.15

In [3]:
### Function to save gridded forcing ###
# 'ts', saved as 'tas' to be consistent with previous files that required Enhui's Matlab Code
def sforc(gpath, vname, fname_out, lat_out, lon_out, time_out, f_out):
    # Save output - this could be a function earlier in the code
    writing = nc.Dataset(gpath + vname + fname_out, "w", format="NETCDF3_64BIT_OFFSET")
    time    = writing.createDimension("time", None)  # sets unlimited dimension
    yh      = writing.createDimension("yh", len(lat_out))
    xh      = writing.createDimension("xh", len(lon_out))

    yh     = writing.createVariable("yh","f8",("yh"))
    yh[:]  = lat_out
    yh.standard_name    = "latitude"
    yh.long_name        = "latitude"
    yh.units            = "degrees_north"
    yh.axis             = "Y"

    xh     = writing.createVariable("xh","f8",("xh"))
    xh[:]  = lon_out
    xh.standard_name    = "longitude"
    xh.long_name        = "longitude"
    xh.units            = "degrees_east"
    xh.axis             = "X" ;

    time    = writing.createVariable("time","f8",("time"))
    time[:] = time_out
    time.standard_name   = "time"
    time.long_name       = "time"
    time.units           = "days since 1900-01-01 00:00:00" ;
    time.calendar        = "gregorian" ;
    time.axis            = "T" ;
    time.modulo          = " " ;

    var          = writing.createVariable(vname,"f8",("time","yh","xh"),fill_value=fillval)
    var[:,:,:]   = f_out

    # Write information about each variable
    # Forcing variables
    if (vname == 'tas'):
        var.long_name      = "Surface Temperature"
        var.units          = "K"
    elif (vname == 'huss'):
        var.long_name      = "Near-Surface Specific Humidity"
        var.units          = "1"
    elif (vname == 'rsds'):    
        var.long_name      = "Surface Downwelling Shortwave Radiation"
        var.units          = "W m-2"
    elif (vname == 'rlds'):    
        var.long_name      = "Surface Downwelling Longwave Radiation"
        var.units          = "W m-2"
    elif (vname == 'prra'):
        var.long_name      = "Rainfall Flux"
        var.units          = "kg m-2 s-1"
    elif (vname == 'prsn'):
        var.long_name      = "Snowfall Flux"
        var.units          = "kg m-2 s-1"

    writing.close()

In [4]:
def low_filter(data, t_coupe):
    
    sig = data
    fe = 1 # Fréquence d'échantillonnage (yr-1)
    f_nyq = fe / 2.  # Fréquence de nyquist
    fc = 1/t_coupe # Fréquence de coupure (yr-1)
    b, a = signal.butter(4, fc/f_nyq, 'low', analog=False) #filtre de Butterworth en passe-bas
    s_but = signal.filtfilt(b, a, sig) # Application du filtre
    
    return s_but

In [5]:
### Grid forcing zonal means by repeating at each point of longitude ###
for iforc in ['rlds']:
    
    # Load appropriate file & create right time vector
    pN = xr.open_dataset(spath + iforc + fnameLF_in) 
    time_out  = time_m  # using day of year
    fname_out = fnameLF_out
    
    # Create a zero array to store the forcing files
    f_out = np.zeros((len(time_out), len(lat_out), len(lon_out)))
             
    # At every time step 
    for itime in np.arange(0,len(time_out)):
        # Interpolate zonal mean to grid latitude
        f0 = np.interp(lat_out, pN.lat, pN[iforc][itime,:])
        #f0 = low_filter(f0, t_coupe = 80)
        f0 = low_filter(f0, t_coupe = 80)+8.5
        
        # Repeat the profile everywhere in our domain
        f_out[itime, :, :] = np.tile(f0.reshape(len(lat_out),1), len(lon_out))

    # There should not be any NaNs, but replace in case
    f_out[np.isnan(f_out)] = fillval  
    
    if (iforc == 'ts'):
        vname = 'tas'
    else: 
        vname = iforc

    # Save output - this could be a function earlier in the code
    sforc(gpath, vname, fname_out, lat_out, lon_out, time_out, f_out)

In [87]:
### Grid forcing zonal means by repeating at each point of longitude ###
for iforc in forcings:
    
    # Load appropriate file & create right time vector
    pN = xr.open_dataset(spath + iforc + fnameLF_in) 
    time_out  = time_m  # using day of year
    fname_out = fnameLF_out
    
    # Create a zero array to store the forcing files
    f_out = np.zeros((len(time_out), len(lat_out), len(lon_out)))
             
    # At every time step 
    for itime in np.arange(0,len(time_out)):
        # Interpolate zonal mean to grid latitude
        f0 = np.interp(lat_out, pN.lat, pN[iforc][itime,:])
        f0 = low_filter(f0, t_coupe = 80)
        
        #### IDEALIZED WARMING
        if (iforc == 'ts'):
            f0 = low_filter(f0, t_coupe = 80)
            scale_lat = (1-np.cos(np.max([np.zeros(len(lat_out)),(lat_out-40)/20], axis = 0)* np.pi/2))
            scale_time = (np.cos((itime-1)/11 * 2 *np.pi)+1)/2
            f0 = f0 - scale_lat * scale_time * 8

        #### WARMING
        #if (iforc == 'ts'):
        #    f0 = low_filter(f0, t_coupe = 80) + 4
        #    print('Temperature increased')
        #if (iforc == 'rlds'):
        #    f0 = low_filter(f0, t_coupe = 80) + 8.5
        #    print('LW increased')
        
        # Repeat the profile everywhere in our domain
        f_out[itime, :, :] = np.tile(f0.reshape(len(lat_out),1), len(lon_out))

    # There should not be any NaNs, but replace in case
    f_out[np.isnan(f_out)] = fillval  
    
    if (iforc == 'ts'):
        vname = 'tas'
    else: 
        vname = iforc

    # Save output - this could be a function earlier in the code
    sforc(gpath, vname, fname_out, lat_out, lon_out, time_out, f_out)

In [88]:
### Grid and scale zonal means associated with precipitation ###
for iforc in scaledF:

    # Load appropriate file & create right time vector
    pN = xr.open_dataset(spath + iforc + fnameLF_in) 
    time_out  = time_m  # using day of year
    fname_out = '_scaled' + fnameLF_out

    # Load the information for scaling the precipitation
    ocedim = xr.open_dataset('./model_ocean_widths_sq.nc')
    rM2O =  ocedim.mwidth / ocedim.owidth # calculate ratio

    # Interpolate scaling to grid latitude
    r0 = np.interp(lat_out, ocedim.lat, rM2O)

    # Create a zero array to store the forcing files
    f_out = np.zeros((len(time_out), len(lat_out), len(lon_out)))

    # At every time step (xarray way of doing so?)
    for itime in np.arange(0,len(time_out)):
        # Interpolate zonal mean to grid latitude
        f0 = np.interp(lat_out, pN.lat, pN[iforc][itime,:])

        # Scale mean precipitation to area in model vs. real ocean
        fS = f0 / r0 # CST PRECIPITATION !!!
        fS = np.full(np.shape(f0), np.mean(fS))

        # Repeat the profile everywhere in our domain
        f_out[itime, :, :] = np.tile(fS.reshape(len(lat_out),1), len(lon_out))

    # There should not be any NaNs, but replace in case
    f_out[np.isnan(f_out)] = fillval  

    vname = iforc
        
    # Save output 
    sforc(gpath, vname, fname_out, lat_out, lon_out, time_out, f_out) 